##  Payment Methods and Their Implementations

In [15]:
class PaymentMethod(ABC):
    @abstractmethod
    def validate_payment_details(self) -> bool:
        """Abstract method to validate payment details."""
        pass

    @abstractmethod
    def process_payment(self, amount: float) -> None:
        """Abstract method to process payment."""
        pass

class CreditCardPayment(PaymentMethod):
    def __init__(self, card_number: str, expiry_date: str, cvv: str):
        self.card_number = card_number
        self.expiry_date = expiry_date
        self.cvv = cvv

    def validate_payment_details(self) -> bool:
        if self._validate_card_number() and self._validate_expiry_date() and self._validate_cvv():
            print(f"Validating credit card details for card ending in {self.card_number[-4:]}...")
            return True
        else:
            print("Invalid credit card details.")
            return False

    def _validate_card_number(self) -> bool:
        return len(self.card_number) == 16 and self.card_number.isdigit()

    def _validate_expiry_date(self) -> bool:
        try:
            month, year = self.expiry_date.split("/")
            if len(month) == 2 and len(year) == 2:
                month = int(month)
                return 1 <= month <= 12
        except ValueError:
            return False
        return False

    def _validate_cvv(self) -> bool:
        return len(self.cvv) == 3 and self.cvv.isdigit()

    def process_payment(self, amount: float) -> None:
        print(f"Processing credit card payment of ${amount} from card ending in {self.card_number[-4:]}")


class PayPalPayment(PaymentMethod):
    def __init__(self, email: str):
        self.email = email

    def validate_payment_details(self) -> bool:
        if "@" in self.email and "." in self.email:
            print(f"Validating PayPal account: {self.email}...")
            return True  
        else:
            print("Invalid PayPal email address.")
            return False

    def process_payment(self, amount: float) -> None:
        print(f"Processing PayPal payment of ${amount} from account {self.email}")

class CryptoPayment(PaymentMethod):
    def __init__(self, wallet_address: str):
        self.wallet_address = wallet_address

    def validate_payment_details(self) -> bool:
        if len(self.wallet_address) == 42 and self.wallet_address.startswith("0x"):
            print(f"Validating cryptocurrency wallet: {self.wallet_address}...")
            return True  
        else:
            print("Invalid cryptocurrency wallet address.")
            return False

    def process_payment(self, amount: float) -> None:
        print(f"Processing cryptocurrency payment of ${amount} from wallet {self.wallet_address}")

## Discount Strategies and Currency Conversion 

In [16]:
class DiscountStrategy(ABC):
    @abstractmethod
    def apply_discount(self, price: float) -> float:
        pass

class PercentageDiscount(DiscountStrategy):
    def __init__(self, percentage: float):
        self.percentage = percentage

    def apply_discount(self, price: float) -> float:
        return price * (1 - self.percentage / 100)

class FixedAmountDiscount(DiscountStrategy):
    def __init__(self, amount: float):
        self.amount = amount

    def apply_discount(self, price: float) -> float:
        return price - self.amount

class CurrencyConverter:
    exchange_rates = {
        "USD": 1.0,
        "EUR": 0.85,
        "GBP": 0.75
    }

    @staticmethod
    def convert(amount: float, from_currency: str, to_currency: str) -> float:
        if from_currency not in CurrencyConverter.exchange_rates or to_currency not in CurrencyConverter.exchange_rates:
            raise ValueError("Unsupported currency")
        return amount / CurrencyConverter.exchange_rates[from_currency] * CurrencyConverter.exchange_rates[to_currency]

class Item:
    def __init__(self, price: float, discount_strategy: DiscountStrategy = None):
        self.price = price
        self.discount_strategy = discount_strategy

    def price_after_discount(self) -> float:
        if self.discount_strategy:
            return self.discount_strategy.apply_discount(self.price)
        return self.price

    def __repr__(self):
        return f"Original Price: ${self.price:.2f}, Price After Discount: ${self.price_after_discount():.2f}"

class PaymentProcessor:
    def __init__(self, payment_method: PaymentMethod, item: Item):
        self.payment_method = payment_method
        self.item = item
        self.db = Database().connect()

    def process_payment(self) -> None:
        if self.payment_method.validate_payment_details():
            final_amount = self.item.price_after_discount()
            self.payment_method.process_payment(final_amount)
            self.log_transaction(final_amount, "Success")
        else:
            print("Payment validation failed.")
            self.log_transaction(0, "Failure")

    def log_transaction(self, amount: float, status: str) -> None:
        Database().log_transaction(amount, status)

## Processing Payments with Different Methods and Logging Transactions

In [17]:
import sqlite3
from abc import ABC, abstractmethod
from datetime import datetime

class MetaSingleton(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]
class Database(metaclass=MetaSingleton):
    connection = None

    def connect(self):
        if self.connection is None:
            self.connection = sqlite3.connect("db.sqlite3")
            self.cursorobj = self.connection.cursor()
        return self.cursorobj

    def create_transaction_log_table(self):
        self.cursorobj.execute('''
        CREATE TABLE IF NOT EXISTS transaction_logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            amount REAL,
            status TEXT,
            timestamp TEXT
        )''')
        self.connection.commit()

    def log_transaction(self, amount: float, status: str):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.cursorobj.execute('''
        INSERT INTO transaction_logs (amount, status, timestamp)
        VALUES (?, ?, ?)
        ''', (amount, status, timestamp))
        self.connection.commit()

    def fetch_all_logs(self):
        self.cursorobj.execute("SELECT * FROM transaction_logs")
        return self.cursorobj.fetchall()

## Running the Application and Viewing Transaction Logs 

In [18]:
if __name__ == "__main__":
    db = Database().connect()
    Database().create_transaction_log_table()
    item1 = Item(200, discount_strategy=PercentageDiscount(20))
    item2 = Item(150, discount_strategy=FixedAmountDiscount(30))
    item3 = Item(100)
    credit_card_payment = CreditCardPayment(
        card_number="1234567812345678", 
        expiry_date="12/25", 
        cvv="123"
    )
    paypal_payment = PayPalPayment(
        email="user@gmail.com"
    )
    crypto_payment = CryptoPayment(
        wallet_address="0x1234567890abcdef1234567890abcdef12345678"
    )

    processor1 = PaymentProcessor(credit_card_payment, item1)
    processor1.process_payment()
    processor2 = PaymentProcessor(paypal_payment, item2)
    processor2.process_payment()
    processor3 = PaymentProcessor(crypto_payment, item3)
    processor3.process_payment()
    logs = Database().fetch_all_logs()
    print("\nTransaction Logs:")
    for log in logs:
        print(log)

Validating credit card details for card ending in 5678...
Processing credit card payment of $160.0 from card ending in 5678
Validating PayPal account: user@example.com...
Processing PayPal payment of $120 from account user@example.com
Validating cryptocurrency wallet: 0x1234567890abcdef1234567890abcdef12345678...
Processing cryptocurrency payment of $100 from wallet 0x1234567890abcdef1234567890abcdef12345678

Transaction Logs:
(1, 160.0, 'Success', '2024-08-21 20:59:04')
(2, 120.0, 'Success', '2024-08-21 20:59:04')
(3, 100.0, 'Success', '2024-08-21 20:59:04')
(4, 160.0, 'Success', '2024-08-22 20:09:11')
(5, 120.0, 'Success', '2024-08-22 20:09:11')
(6, 100.0, 'Success', '2024-08-22 20:09:11')
(7, 160.0, 'Success', '2024-08-22 20:10:11')
(8, 120.0, 'Success', '2024-08-22 20:10:11')
(9, 100.0, 'Success', '2024-08-22 20:10:11')
(10, 160.0, 'Success', '2024-08-22 21:20:46')
(11, 120.0, 'Success', '2024-08-22 21:20:46')
(12, 100.0, 'Success', '2024-08-22 21:20:46')
(13, 160.0, 'Success', '20